In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import Lasso, Ridge, LinearRegression
pd.options.display.max_columns = 0

In [2]:
data = pd.read_csv("../data/processed/tennisResults.csv")
data.head()

,tourney_name,surface,tourney_level,winner_id,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,United Cup,Hard,A,126203,R,193.0,USA,25.1,126610,R,196.0,ITA,26.7,3,F,135.0,15.0,2.0,85.0,52.0,45.0,16.0,12.0,0.0,0.0,7.0,2.0,97.0,62.0,47.0,15.0,12.0,9.0,9.0
1,United Cup,Hard,A,126207,R,188.0,USA,24.9,207518,R,185.0,ITA,20.8,3,F,34.0,3.0,1.0,27.0,19.0,15.0,3.0,4.0,3.0,3.0,0.0,0.0,21.0,12.0,8.0,3.0,4.0,1.0,3.0
2,United Cup,Hard,A,126203,R,193.0,USA,25.1,128034,R,196.0,POL,25.8,3,SF,104.0,11.0,0.0,80.0,50.0,44.0,19.0,12.0,4.0,4.0,17.0,1.0,82.0,62.0,51.0,7.0,12.0,2.0,2.0
3,United Cup,Hard,A,126207,R,188.0,USA,24.9,200390,R,183.0,POL,23.9,3,SF,79.0,5.0,4.0,61.0,37.0,31.0,11.0,9.0,5.0,5.0,3.0,3.0,68.0,41.0,26.0,12.0,9.0,6.0,9.0
4,United Cup,Hard,A,126774,R,193.0,GRE,24.3,126610,R,196.0,ITA,26.7,3,SF,155.0,9.0,1.0,98.0,52.0,43.0,28.0,16.0,1.0,2.0,7.0,2.0,89.0,58.0,48.0,18.0,16.0,1.0,2.0


In [3]:
columns_LE = ["tourney_name", "winner_ioc", "loser_ioc", "round"]
le = LabelEncoder()

data["tourney_name"] = le.fit_transform(data["tourney_name"])
data["winner_ioc"] = le.fit_transform(data["winner_ioc"])
data["loser_ioc"] = le.fit_transform(data["loser_ioc"])
data["round"] = le.fit_transform(data["round"])
data.head()

,tourney_name,surface,tourney_level,winner_id,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,46,Hard,A,126203,R,193.0,43,25.1,126610,R,196.0,28,26.7,3,0,135.0,15.0,2.0,85.0,52.0,45.0,16.0,12.0,0.0,0.0,7.0,2.0,97.0,62.0,47.0,15.0,12.0,9.0,9.0
1,46,Hard,A,126207,R,188.0,43,24.9,207518,R,185.0,28,20.8,3,0,34.0,3.0,1.0,27.0,19.0,15.0,3.0,4.0,3.0,3.0,0.0,0.0,21.0,12.0,8.0,3.0,4.0,1.0,3.0
2,46,Hard,A,126203,R,193.0,43,25.1,128034,R,196.0,41,25.8,3,7,104.0,11.0,0.0,80.0,50.0,44.0,19.0,12.0,4.0,4.0,17.0,1.0,82.0,62.0,51.0,7.0,12.0,2.0,2.0
3,46,Hard,A,126207,R,188.0,43,24.9,200390,R,183.0,41,23.9,3,7,79.0,5.0,4.0,61.0,37.0,31.0,11.0,9.0,5.0,5.0,3.0,3.0,68.0,41.0,26.0,12.0,9.0,6.0,9.0
4,46,Hard,A,126774,R,193.0,23,24.3,126610,R,196.0,28,26.7,3,7,155.0,9.0,1.0,98.0,52.0,43.0,28.0,16.0,1.0,2.0,7.0,2.0,89.0,58.0,48.0,18.0,16.0,1.0,2.0


In [4]:
map_surface = {"Hard":0, "Clay": 1, "Grass":2}
map_hand = {"R":0, "L": 1, "U":2}
map_tourney = {"A":0, "M": 1, "G":2}

data["surface"].replace(map_surface, inplace=True)
data["loser_hand"].replace(map_hand, inplace=True)
data["winner_hand"].replace(map_hand, inplace=True)
data["tourney_level"].replace(map_tourney, inplace=True)


In [5]:
data.head()

,tourney_name,surface,tourney_level,winner_id,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,46,0,0,126203,0,193.0,43,25.1,126610,0,196.0,28,26.7,3,0,135.0,15.0,2.0,85.0,52.0,45.0,16.0,12.0,0.0,0.0,7.0,2.0,97.0,62.0,47.0,15.0,12.0,9.0,9.0
1,46,0,0,126207,0,188.0,43,24.9,207518,0,185.0,28,20.8,3,0,34.0,3.0,1.0,27.0,19.0,15.0,3.0,4.0,3.0,3.0,0.0,0.0,21.0,12.0,8.0,3.0,4.0,1.0,3.0
2,46,0,0,126203,0,193.0,43,25.1,128034,0,196.0,41,25.8,3,7,104.0,11.0,0.0,80.0,50.0,44.0,19.0,12.0,4.0,4.0,17.0,1.0,82.0,62.0,51.0,7.0,12.0,2.0,2.0
3,46,0,0,126207,0,188.0,43,24.9,200390,0,183.0,41,23.9,3,7,79.0,5.0,4.0,61.0,37.0,31.0,11.0,9.0,5.0,5.0,3.0,3.0,68.0,41.0,26.0,12.0,9.0,6.0,9.0
4,46,0,0,126774,0,193.0,23,24.3,126610,0,196.0,28,26.7,3,7,155.0,9.0,1.0,98.0,52.0,43.0,28.0,16.0,1.0,2.0,7.0,2.0,89.0,58.0,48.0,18.0,16.0,1.0,2.0


In [6]:
data.describe().round(2)

,tourney_name,surface,tourney_level,winner_id,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
count,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00,2129.00
mean,26.79,0.71,0.61,142017.71,0.14,187.37,23.13,26.69,140857.17,0.19,186.55,25.97,27.10,3.36,3.48,116.00,6.67,2.57,79.15,49.97,37.73,16.20,12.67,3.44,4.98,5.21,3.24,81.77,50.43,33.82,14.62,12.46,4.76,8.42
std,15.07,0.72,0.77,41518.86,0.38,6.18,13.86,4.26,41230.43,0.45,6.21,17.26,4.32,0.77,1.55,39.73,5.06,2.21,29.22,19.61,13.97,6.53,4.23,3.06,4.01,4.93,2.61,28.99,19.27,14.46,6.81,4.21,3.27,4.13
min,0.00,0.00,0.00,100644.00,0.00,170.00,0.00,17.90,100644.00,0.00,170.00,0.00,16.50,3.00,0.00,5.00,0.00,0.00,5.00,3.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,4.00,3.00,0.00,0.00,1.00,0.00,0.00
25%,13.00,0.00,0.00,106296.00,0.00,183.00,14.00,24.10,106214.00,0.00,183.00,13.00,24.20,3.00,2.00,87.00,3.00,1.00,58.00,36.00,28.00,12.00,10.00,1.00,2.00,2.00,1.00,60.00,36.00,24.00,10.00,9.00,2.00,5.00
50%,28.00,1.00,0.00,126203.00,0.00,187.37,22.00,26.20,126094.00,0.00,185.00,22.00,26.60,3.00,4.00,113.00,6.00,2.00,75.00,47.00,35.00,15.00,12.00,3.00,4.00,4.00,3.00,77.00,47.00,31.00,14.00,12.00,4.00,8.00
75%,40.00,1.00,1.00,200221.00,0.00,191.00,37.00,28.90,200175.00,0.00,191.00,45.00,29.70,3.00,4.00,138.00,9.00,4.00,95.00,60.00,45.00,20.00,15.00,5.00,7.00,7.00,4.00,98.00,61.00,42.00,18.00,15.00,7.00,11.00
max,50.00,2.00,2.00,210506.00,2.00,206.00,43.00,41.70,212021.00,2.00,206.00,53.00,41.70,5.00,7.00,345.00,42.00,15.00,228.00,142.00,100.00,45.00,31.00,22.00,26.00,44.00,25.00,205.00,143.00,101.00,47.00,31.00,19.00,28.00


# División de datos y Train-Test

In [7]:
data.columns

Index(['tourney_name', 'surface', 'tourney_level', 'winner_id', 'winner_hand',
       'winner_ht', 'winner_ioc', 'winner_age', 'loser_id', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'best_of', 'round', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn',
       'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced'],
      dtype='object')

In [8]:
X = data.drop(columns=["minutes", "tourney_name", "surface","tourney_level",'winner_ioc','loser_ioc','winner_ht','loser_ht'], axis=1)
X = np.log(X + 1)
y = data["minutes"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 42)

# Cross_validation + Base Lines

In [23]:
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


modelos = {
    "Linear Regression": LinearRegression(),
   "Ridge": Ridge(alpha=0.1, solver="cholesky"),
    "Lasso": Lasso(alpha=0.1),
    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5),
    "LinearSVR": LinearSVR(epsilon=0.5, dual=True, random_state=42),
    "DecissionTree":DecisionTreeRegressor(max_depth=2, random_state=42),
    "RandomForest":RandomForestRegressor(n_estimators=10, random_state=42)
}

# Define las métricas a usar
metricas = ["neg_mean_absolute_percentage_error", "neg_root_mean_squared_error"]

resultados_dict = {}

for nombre_modelo, modelo in modelos.items():
    cv_resultados = cross_validate(modelo, X_train, y_train, cv=5, scoring=metricas)
    
    for metrica in metricas:
        
        clave = f"{nombre_modelo}_{metrica}"
        resultados_dict[clave] = cv_resultados[f"test_{metrica}"].mean()
        
        # metrica = f"{metrica}"
        # resultados_dict["modelo"] = nombre_modelo
        # resultados_dict["metrica"] = cv_resultados[f"test_{metrica}"].mean()
       

# Convertir el diccionario de resultados en DataFrame
resultados_df = pd.DataFrame([resultados_dict])

c:\Users\jaime\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\jaime\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\jaime\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\jaime\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\jaime\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warni

In [13]:
data["minutes"].mean().round(2)

116.0

In [31]:
abs(resultados_df.T).sort_values(by=0, ascending=True)

,0
RandomForest_neg_mean_absolute_percentage_error,0.099385
Ridge_neg_mean_absolute_percentage_error,0.138925
Linear Regression_neg_mean_absolute_percentage_error,0.138967
Lasso_neg_mean_absolute_percentage_error,0.139125
ElasticNet_neg_mean_absolute_percentage_error,0.139211
LinearSVR_neg_mean_absolute_percentage_error,0.140623
DecissionTree_neg_mean_absolute_percentage_error,0.171127
RandomForest_neg_root_mean_squared_error,18.578910
Ridge_neg_root_mean_squared_error,19.855169
Linear Regression_neg_root_mean_squared_error,19.857553


In [32]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True,False],
    'max_depth': [50, 80],
    'max_features': [15, 30],
    'min_samples_leaf': [3,5],
    'min_samples_split': [8, 10],
    'n_estimators': [500, 1000]
}

elastic = RandomForestRegressor()

grid_search = GridSearchCV(elastic,
                           param_grid,
                           cv=5,
                           scoring='neg_mean_absolute_percentage_error',
                           n_jobs=-1
                          )
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [50, 80],
                         'max_features': [15, 30], 'min_samples_leaf': [3, 5],
                         'min_samples_split': [8, 10],
                         'n_estimators': [500, 1000]},
             scoring='neg_mean_absolute_percentage_error')

In [33]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 50,
 'max_features': 15,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [34]:
rforest = RandomForestRegressor(bootstrap= True, max_depth= 80, max_features= 15, min_samples_leaf= 3, min_samples_split= 8, n_estimators= 1000)
rforest.fit(X_train, y_train)

RandomForestRegressor(max_depth=80, max_features=15, min_samples_leaf=3,
                      min_samples_split=8, n_estimators=1000)

In [35]:
rforest.score(X_test, y_test)

0.8235313003700641

In [36]:
from sklearn.metrics import mean_absolute_percentage_error

pred = rforest.predict(X_test)
mean_absolute_percentage_error(y_test, pred)

0.08769458813705557

In [48]:
dataframe = pd.read_csv("../data/processed/tennisRes.csv", sep = ";")
dataframe.head()

,winner_id,winner_hand,winner_age,loser_id,loser_hand,loser_age,best_of,round,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,126203,R,25.1,126610,R,26.7,3,F,15.0,2.0,85.0,52.0,45.0,16.0,12.0,0.0,0.0,7.0,2.0,97.0,62.0,47.0,15.0,12.0,9.0,9.0
1,126207,R,24.9,207518,R,20.8,3,F,3.0,1.0,27.0,19.0,15.0,3.0,4.0,3.0,3.0,0.0,0.0,21.0,12.0,8.0,3.0,4.0,1.0,3.0
2,126203,R,25.1,128034,R,25.8,3,SF,11.0,0.0,80.0,50.0,44.0,19.0,12.0,4.0,4.0,17.0,1.0,82.0,62.0,51.0,7.0,12.0,2.0,2.0
3,126207,R,24.9,200390,R,23.9,3,SF,5.0,4.0,61.0,37.0,31.0,11.0,9.0,5.0,5.0,3.0,3.0,68.0,41.0,26.0,12.0,9.0,6.0,9.0
4,126774,R,24.3,126610,R,26.7,3,SF,9.0,1.0,98.0,52.0,43.0,28.0,16.0,1.0,2.0,7.0,2.0,89.0,58.0,48.0,18.0,16.0,1.0,2.0


In [49]:
dataframe["round"] = le.fit_transform(dataframe["round"])

map_hand = {"R":0, "L": 1, "U":2}

dataframe["loser_hand"].replace(map_hand, inplace=True)
dataframe["winner_hand"].replace(map_hand, inplace=True)

In [50]:
rforest.predict(dataframe)

array([200.27159952, 229.83960477, 195.58395514, 184.68232104,
       195.69785293, 197.78624681, 197.4823429 , 198.00696683,
       200.50720424, 189.03192258, 193.77776413, 200.64387673])

In [44]:
pred

array([188.09138247,  92.57498748,  65.08207226, 133.95676296,
       129.42886106, 119.36782881,  68.70497532,  96.51000015,
        86.62699715, 116.82432949, 143.55913818,  75.06640491,
        90.68340667,  67.19976032,  72.17467399, 103.17258966,
       141.6233528 ,  68.97323524, 164.81735776, 115.01155854,
       165.28669157, 169.54888373,  60.17896736, 116.01328642,
       127.44904671, 128.43012327, 171.6535664 , 155.26864758,
       141.57158207, 104.89118589, 138.89516666,  85.83278173,
        86.11792653, 130.2800893 ,  61.38669631, 147.68301691,
       137.49807077, 111.90826936, 129.84836427, 149.41772461,
       119.65219876, 105.41977652, 177.38995888, 166.76746448,
       135.66833987, 194.78101855, 119.23488153, 172.43451654,
        66.85982766, 167.92879194,  86.70763638, 125.00971011,
       105.9114234 , 150.12554467, 109.57796173, 182.2098033 ,
        93.30204239,  49.05730341, 148.94991165,  41.22393027,
        78.64443896,  90.59615389, 136.23517176,  86.51

In [37]:
grid_search.best_score_

-0.09462836615901916

In [38]:
grid_search.best_estimator_.score(X_test, y_test)

0.8234279187544518

In [39]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X_train)

In [40]:
pca.score(X_test, y_test)

-9.922382461338312